In [5]:
from dotenv import load_dotenv

load_dotenv('.env.dev')

True

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_upstage import (
    UpstageLayoutAnalysisLoader,
    UpstageGroundednessCheck,
    ChatUpstage,
    UpstageEmbeddings,
)
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")

In [9]:
vectorstore = Chroma(persist_directory='chromadb.sqlite', embedding_function=embeddings_model)


# vectorstore = Chroma.from_documents(
#     documents=documents[0:100], 
#     embedding=embeddings_model, 
#     # persist_directory="./chroma_db"
# )

In [10]:
from langchain_community.llms.solar import Solar
# {doc.metadata["유사 영화 내용"]}
def invoke_form(doc): 
    content = f"""
    <제목>
    {doc.metadata["영화 제목"]}
    </제목>

    <감독>
    {doc.metadata["영화 감독"]}
    </감독>

    <등장인물>
    {doc.metadata["영화 등장인물"]}
    </등장인물>
    
    <줄거리>
    {doc.metadata["영화 줄거리"]}
    </줄거리>

    <유사 영화>
    {doc.metadata["유사 영화"]}
    </유사 영화>
    """
    return content 
"""
1. 90 Minutes in Heaven (2015) - 이 영화는 교통사고 후 천국을 경험한 남자의 이야기를 그린 작품으로, 범죄도시2와는 다른 장르이지만 감동적인 이야기를 담고 있습니다.
2. Noise (2007) - 이 영화는 소음으로 인해 괴로워하는 사람들의 이야기를 그린 작품으로, 범죄도시2와는 다른 분위기를 가지고 있지만 사회적인 문제를 다루고 있습니다.
3. Good Night, and Good Luck. (2005) - 이 영화는 1950년대 미국의 언론인과 그의 동료들이 언론의 자유를 위해 싸우는 이야기를 그린 작품으로, 범죄도시2와는 다른 시대와 배경을 가지고 있지만 사회적인 메시지를 담고 있습니다.
4. The Bulwark (2019) - 이 영화는 범죄도시2와 마찬가지로 범죄와 싸우는 경찰의 이야기를 그린 작품으로, 범죄도시2와 비슷한 분위기를 가지고 있습니다.
5. Mulan: Rise of a Warrior (2009) - 이 영화는 중국 전통 이야기를 바탕으로 한 애니메이션으로, 범죄도시2와는 다른 장르이지만 용기와 희생에 대한 이야기를 담고 있습니다.
"""
def format_docs(docs):
    return "\n\n".join(invoke_form(doc) for doc in docs[0:1])

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"k": 1, "score_threshold": 0.01}) # Query와 유사한거 찾는 녀석 (K=4)

In [12]:
template_rec = """
당신은 추천시스템 서비스2 모임에서 만들어진 영화 추천을 도와주는 AI Chatbot인 PseudoRec입니다.
주어진 <영화정보>를 바탕으로 유사한 영화에 대해 알려주세요. 
만약, 모르는 영화인 경우 "검색"을 출력하세요. 

<영화정보> 
{context}
</영화정보> 

<질문> 
{question}
</질문> 
"""

template_search = """
당신은 추천시스템 서비스2 모임에서 만들어진 영화 추천을 도와주는 AI Chatbot인 PseudoRec입니다.

<내용>
{context}
</내용>

<질문> 
{question}
</질문>

주어진 <내용>을 기반으로 사용자의 <질문>에 답변을 해주세요. 
"""

template_chat = """
당신은 추천시스템 서비스2 모임에서 만들어진 영화 추천을 도와주는 AI Chatbot인 PseudoRec입니다.
친절하고, 상냥하고, 존댓말로 사용자의 질문에 답변을 해주세요. 

<질문> 
{question}
</질문>
"""

custom_rec_prompt = PromptTemplate.from_template(template_rec)
custom_search_prompt = PromptTemplate.from_template(template_search)
custom_chat_prompt = PromptTemplate.from_template(template_chat)

In [17]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rec_prompt # prompt
    | ChatUpstage() # chat
    | StrOutputParser() # output parser
)

question = '범죄도시와 비슷한 영화 추천해줘'
question = '너 누구야'
new_response = rag_chain.invoke(question)

/Users/kyeongchanlee/PycharmProjects/recsys_service_deployment/venv/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.01
  warnings.warn(


In [19]:
new_response

'죄송합니다, 하지만 제공된 정보가 없어서 당신을 도와드릴 수 없습니다. 영화 정보를 제공해주실 수 있을까요?'

In [20]:
import os

In [23]:
import boto3

In [24]:
s3 = boto3.client('s3', 
                       aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'), 
                       aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'))

In [29]:
s3.upload_file('chroma.sqlite3', 'pseudorec-data', 'hyeonwoo/chroma.sqlite3')